In [7]:
# Date Packages
from datetime import date, datetime

# Database Packages
import psycopg2

# Yahoo Finance Packages
import yfinance as yf

# Timeout Packages
from interruptingcow import timeout

# Data Processing Packages
import pandas as pd
from itertools import cycle

import sys
sys.path.append('../utils')

from db_operator import Db_Operator

## TaskID

In [8]:
taskid = '{date:%Y%m%d-%H%M%S}'.format( date=datetime.now()) 
print(taskid)

20190725-084742


In [9]:
dbo = Db_Operator(taskid)

In [10]:
proxies = dbo.get_proxy()
proxy_pool = cycle(proxies)

df_stock_master = dbo.show_stock_master()

## Download Stock Price Data

In [11]:
# Select Stocks
df_stock = df_stock_master[df_stock_master.industryclassification =='銀行']
stocks = df_stock['stockcode'].tolist()

# Select Time period
start = datetime(2018,1,1)
end = datetime.today()

# Create Empty Stock Price Table
col_names = ['Stock Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
df_stock_price = pd.DataFrame(columns = col_names)

# Download Error Message
error_msg = ''

In [12]:
# Get new proxy
proxy = next(proxy_pool)

# Get stock price from selected stocks
for stock in stocks:
    
    isDownloaded = False
    error_count = 0
   
    # Download stock
    while isDownloaded == False and error_count < 50:
        try:

            print('--------------------------------------------------------')
            print('stock: ', stock, ' using proxy: ', proxy)
            
            # Download within 5 seconds
            with timeout(10, exception=RuntimeError):
                data = yf.download(tickers=stock, start=start, end=end, proxy=proxy)
                isDownloaded = True
        except:
            print("Proxy Error")
            proxy = next(proxy_pool)
            error_count = error_count + 1
    
    # Check Download Success
    if isDownloaded == True:
        data['Date'] = data.index
        data['Stock Code'] = stock
        df_stock_price = df_stock_price.append(data, sort=False)
    else:
        error_msg = error_msg + format('[Error downloading %s]',stock)
        
df_stock_price = df_stock_price[col_names]

--------------------------------------------------------
stock:  3866.HK  using proxy:  34.90.137.234:80
[*********************100%***********************]  1 of 1 downloaded
--------------------------------------------------------
stock:  3968.HK  using proxy:  34.90.137.234:80
[*********************100%***********************]  1 of 1 downloaded
--------------------------------------------------------
stock:  3988.HK  using proxy:  34.90.137.234:80
[*********************100%***********************]  1 of 1 downloaded
--------------------------------------------------------
stock:  6196.HK  using proxy:  34.90.137.234:80
[*********************100%***********************]  1 of 1 downloaded
--------------------------------------------------------
stock:  6122.HK  using proxy:  34.90.137.234:80
[*********************100%***********************]  1 of 1 downloaded
--------------------------------------------------------
stock:  6138.HK  using proxy:  34.90.137.234:80
[*******************

## Insert Stock Price

In [13]:
df_stock_price.head()

,Stock Code,Date,Open,High,Low,Close,Adj Close,Volume
2018-01-02,3866.HK,2018-01-02,6.94,6.94,6.94,6.94,6.439,0
2018-01-03,3866.HK,2018-01-03,6.90,6.90,6.90,6.90,6.402,2500
2018-01-04,3866.HK,2018-01-04,6.65,6.65,6.65,6.65,6.170,500
2018-01-05,3866.HK,2018-01-05,6.65,6.65,6.63,6.65,6.170,3000
2018-01-08,3866.HK,2018-01-08,6.65,6.66,6.65,6.65,6.170,64000


In [14]:
try:
    # Delete original stock price data
    for stock in df_stock_price['Stock Code'].unique():
        dbo.delete_stock_price(stock)
        
    # Insert stock price data
    for index, row in df_stock_price.iterrows():
        dbo.add_stock_price(row['Stock Code'], row['Date'], row['Open'], 
                        row['High'], row['Low'], row['Close'], 
                        row['Adj Close'], row['Volume'])
        
    dbo.add_sys_log('add stock price', 'success', error_msg)
except Exception as e:
    dbo.add_sys_log('add stock price', 'fail', str(e))